<a href="https://colab.research.google.com/github/sathishkumar19/Sathish_Portfolio/blob/main/employee_db_hr_dataset_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Employee - HR Database - SQL Analysis

Importing Data for Analysis

In [1]:
from google.colab import auth
from google.cloud import bigquery
import os
import zipfile

In [2]:
# ✅ Authenticate with your Google account
auth.authenticate_user()

# ✅ Ask for project ID
project_id = input("Please enter your Google Cloud Project ID:")

Please enter your Google Cloud Project ID:sathish-scaler-projects


In [3]:
# ✅ Upload your ZIP
from google.colab import files
uploaded = files.upload()

Saving hr.zip to hr.zip


In [37]:
# ✅ Unzip uploaded file
import shutil
zip_file_name = next(iter(uploaded))
base_extract_path = "/content/bq_dataset_files"
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(base_extract_path)

In [38]:
ls /content/bq_dataset_files

countries.csv    employees.csv    jobs.csv       __MACOSX/
departments.csv  job_history.csv  locations.csv  regions.csv


In [44]:
dataset_mapping = {
    "employee_db": ""
}

In [45]:
base_extract_path = "/content/bq_dataset_files"

for dataset_id, subfolder in dataset_mapping.items():
  folder_path = os.path.join(base_extract_path, subfolder)


In [46]:
# ✅ Create datasets and load CSVs
import os
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

for dataset_id, subfolder in dataset_mapping.items():
    dataset_ref = bigquery.Dataset(f"{project_id}.{dataset_id}")
    try:
        client.get_dataset(dataset_ref)
        print(f"✅ Dataset already exists: {dataset_id}")
    except Exception:
        client.create_dataset(dataset_ref)
        print(f"🆕 Created dataset: {dataset_id}")

    # Navigate to the directory containing the CSV files
    folder_path = os.path.join(base_extract_path, subfolder) # Corrected the path

    for file in os.listdir(folder_path):
        # Check if the file ends with .csv
        if file.endswith(".csv"):
            # Extract table_id from the filename by removing .csv and prefix
            table_id = file.replace(".csv", "").split("_", 1)[-1]  # Extract the table name after the underscore
            file_path = os.path.join(folder_path, file)
            table_ref = f"{project_id}.{dataset_id}.{table_id}"  # Use dataset_id here

            job_config = bigquery.LoadJobConfig(
                source_format=bigquery.SourceFormat.CSV,
                skip_leading_rows=1,
                autodetect=True
            )

            with open(file_path, "rb") as source_file:
                job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
            job.result()  # Wait for the job to complete
            print(f"📥 Uploaded: {file} → {table_ref}")

print("✅ All CSV files uploaded to BigQuery.")

✅ Dataset already exists: employee_db
📥 Uploaded: regions.csv → sathish-scaler-projects.employee_db.regions
📥 Uploaded: employees.csv → sathish-scaler-projects.employee_db.employees
📥 Uploaded: locations.csv → sathish-scaler-projects.employee_db.locations
📥 Uploaded: jobs.csv → sathish-scaler-projects.employee_db.jobs
📥 Uploaded: departments.csv → sathish-scaler-projects.employee_db.departments
📥 Uploaded: job_history.csv → sathish-scaler-projects.employee_db.history
📥 Uploaded: countries.csv → sathish-scaler-projects.employee_db.countries
✅ All CSV files uploaded to BigQuery.


In [47]:
from google.cloud import bigquery
from tabulate import tabulate

In [48]:
# Construct a BigQuery client object.
client = bigquery.Client(project='sathish-scaler-projects') # Assuming you have 'project_id' from before

In [49]:
# Construct a SQL query.
query = """
    SELECT *
    FROM `sathish-scaler-projects.employee_db.employees`
    LIMIT 20
    # Replace with your actual dataset and table names
"""
# Execute the query.
query_job = client.query(query)

# Get the results.
results = query_job.result()

# Convert results to pandas DataFrame
df = results.to_dataframe()

# Display DataFrame as a nicely formatted table using tabulate
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+---------------+--------------+-------------+----------+----------------+-------------+------------+----------+------------------+--------------+-----------------+
|    |   employee_id | first_name   | last_name   | email    | phone_number   | hire_date   | job_id     |   salary | commission_pct   | manager_id   |   department_id |
|----+---------------+--------------+-------------+----------+----------------+-------------+------------+----------+------------------+--------------+-----------------|
|  0 |           100 | Steven       | King        | SKING    | 515.123.4567   | 1987-06-17  | AD_PRES    |    25000 | NULL             | NULL         |              90 |
|  1 |           101 | Neena        | Kochhar     | NKOCHHAR | 515.123.4568   | 1989-09-21  | AD_VP      |    17000 | NULL             | 100          |              90 |
|  2 |           102 | Lex          | De Haan     | LDEHAAN  | 515.123.4569   | 1993-01-13  | AD_VP      |    17000 | NULL             | 100          

In [50]:
# Assuming this is within a larger Python script using a database connector
# Example using sqlite3:
import sqlite3
conn = sqlite3.connect('sathish-scaler-projects.cineflix_store.employees')
cursor = conn.cursor()

Q. Which location does all employees belong

In [ ]:
query = """
SELECT *
FROM `sathish-scaler-projects.employee_db.employees`

"""
query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
print(tabulate(df, headers='keys', tablefmt='psql'))



List all the tables with thier schema structure in employees_db


In [55]:
# prompt: List all the tables with thier schema structure in employees_db

from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project='sathish-scaler-projects') # Replace with your project ID

# Replace with your actual dataset ID
dataset_id = 'employee_db'

# Get the dataset
dataset_ref = client.get_dataset(dataset_id)
tables = list(client.list_tables(dataset_ref))  # Make this a list

print(f"Tables in dataset '{dataset_id}':")
for table in tables:
    print(f"Table: {table.table_id}")

    # Get the table schema
    table = client.get_table(table.reference)
    print("Schema:")
    for schema_field in table.schema:
      print(f"  - Name: {schema_field.name}, Type: {schema_field.field_type}, Mode: {schema_field.mode}, Description: {schema_field.description}")
    print("-" * 20)


Tables in dataset 'employee_db':
Table: countries
Schema:
  - Name: country_id, Type: STRING, Mode: NULLABLE, Description: None
  - Name: country_name, Type: STRING, Mode: NULLABLE, Description: None
  - Name: region_id, Type: INTEGER, Mode: NULLABLE, Description: None
--------------------
Table: departments
Schema:
  - Name: department_id, Type: INTEGER, Mode: NULLABLE, Description: None
  - Name: department_name, Type: STRING, Mode: NULLABLE, Description: None
  - Name: manager_id, Type: STRING, Mode: NULLABLE, Description: None
  - Name: location_id, Type: INTEGER, Mode: NULLABLE, Description: None
--------------------
Table: employees
Schema:
  - Name: employee_id, Type: INTEGER, Mode: NULLABLE, Description: None
  - Name: first_name, Type: STRING, Mode: NULLABLE, Description: None
  - Name: last_name, Type: STRING, Mode: NULLABLE, Description: None
  - Name: email, Type: STRING, Mode: NULLABLE, Description: None
  - Name: phone_number, Type: STRING, Mode: NULLABLE, Description: No

![image.png](
  
)

In [65]:
# prompt: Create a Entity Relationship Diagram for the employees_db and place using mermaid in a visual format

erDiagram
    employees {
        employee_id INT PK
        first_name VARCHAR
        last_name VARCHAR
        email VARCHAR
        phone_number VARCHAR
        hire_date DATE
        job_id INT
        salary DECIMAL
        commission_pct DECIMAL
        manager_id INT
        department_id INT
    }

    jobs {
        job_id INT PK
        job_title VARCHAR
        min_salary DECIMAL
        max_salary DECIMAL
    }

    departments {
        department_id INT PK
        department_name VARCHAR
        location_id INT
    }

    locations {
        location_id INT PK
        street_address VARCHAR
        postal_code VARCHAR
        city VARCHAR
        state_province VARCHAR
        country_id VARCHAR
    }

    countries {
        country_id VARCHAR PK
        country_name VARCHAR
        region_id INT
    }

    regions {
        region_id INT PK
        region_name VARCHAR
    }

    employees ||--o{ jobs : works_as
    employees ||--o{ departments : works_in
    departments ||--o{ locations : located_in
    locations ||--o{ countries : belongs_to
    countries ||--o{ regions : part_of
    employees ||--o{ employees : managed_by

IndentationError: unexpected indent (<ipython-input-65-f1113c2816e9>, line 4)

# prompt: Create a Entity Relationship Diagram for the employees_db and place using mermaid

erDiagram
    employees {
        emp_id INT PK
        first_name VARCHAR
        last_name VARCHAR
        email VARCHAR
        phone_number VARCHAR
        hire_date DATE
```mermaid
erDiagram
    employees {
        emp_id INT PK
        first_name VARCHAR
        last_name VARCHAR
        email VARCHAR
        phone_number VARCHAR
        hire_date DATE
        job_id INT
        salary DECIMAL
        commission_pct DECIMAL
        manager_id INT
        department_id INT
    }

    departments {
        department_id INT PK
        department_name VARCHAR
        location_id INT
    }

    jobs {
        job_id INT PK
        job_title VARCHAR
        min_salary DECIMAL
        max_salary DECIMAL
    }

    locations {
        location_id INT PK
        street_address VARCHAR
        postal_code VARCHAR
        city VARCHAR
        state_province VARCHAR
        country_id CHAR
    }
    
    countries {
        country_id CHAR PK
        country_name VARCHAR
        region_id INT
    }

    regions {
        region_id INT PK
        region_name VARCHAR
    }
    

    employees } *--{ departments : works_in
    employees } *--{ jobs : assigned_to
    employees } *--{ employees : manages
    departments } *--{ locations : located_in
    locations } *--{ countries : resides_in
    countries } *--{ regions : belongs_to
```
    }

    departments {
        department_id INT PK
        department_name VARCHAR
        location_id INT
    }

    jobs {
        job_id INT PK
        job_title VARCHAR
        min_salary DECIMAL
        max_salary DECIMAL
    }

    locations {
        location_id INT PK
        street_address VARCHAR
        postal_code VARCHAR
        city VARCHAR
        state_province VARCHAR
        country_id CHAR
    }
    
    countries {
        country_id CHAR PK
        country_name VARCHAR
        region_id INT
    }

    regions {
        region_id INT PK
        region_name VARCHAR
    }
    

    employees } *--{ departments : works_in
    employees } *--{ jobs : assigned_to
    employees } *--{ employees : manages
    departments } *--{ locations : located_in
    locations } *--{ countries : resides_in
    countries } *--{ regions : belongs_to

In [66]:
# prompt: Write a query to list all employees who earn above the average salary of thier department. display each employees, employee_id, department_id, salary and the department's average salary , rounded to the nearest integer


In [67]:
# prompt: Write a query to list all employees who earn above the average salary of thier department.
#display each employees, employee_id, department_id, salary and the department's average salary , rounded to the nearest integer

# Construct a BigQuery client object.
client = bigquery.Client(project='sathish-scaler-projects')

query = """
    SELECT

        e.employee_id,
        e.department_id,
        e.salary,
        ROUND(avg_salary.avg_dept_salary, 0) AS avg_department_salary
    FROM
        `sathish-scaler-projects.employee_db.employees` AS e
    JOIN (
        SELECT
            department_id,
            AVG(salary) AS avg_dept_salary
        FROM
            `sathish-scaler-projects.employee_db.employees`
        GROUP BY
            department_id
    ) AS avg_salary ON e.department_id = avg_salary.department_id
    WHERE e.salary > avg_salary.avg_dept_salary;
"""

query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
print(tabulate(df, headers='keys', tablefmt='psql'))


+----+---------------+-----------------+----------+-------------------------+
|    |   employee_id |   department_id |   salary |   avg_department_salary |
|----+---------------+-----------------+----------+-------------------------|
|  0 |           100 |              90 |    25000 |                   19667 |
|  1 |           103 |              60 |     9000 |                    5760 |
|  2 |           104 |              60 |     6000 |                    5760 |
|  3 |           108 |             100 |    12000 |                    8600 |
|  4 |           109 |             100 |     9000 |                    8600 |
|  5 |           114 |              30 |    11000 |                    4150 |
|  6 |           120 |              50 |     8000 |                    3476 |
|  7 |           121 |              50 |     8200 |                    3476 |
|  8 |           122 |              50 |     7900 |                    3476 |
|  9 |           123 |              50 |     6500 |             

In [68]:
# Prompt : Write a query to display details of employees who have the 3rd highest salary in each department, if multiple emplouyees have the same salary as the 3rd highest in the department in a deaprtment, include all of them. the output should include department_id, employee_id, employee_name(fullname), salary

query = """
    SELECT
        department_id,
        employee_id,
        CONCAT(first_name, ' ', last_name) AS employee_name,
        salary
    FROM (

"""

query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
print(tabulate(df, headers='keys', tablefmt='psql'))

In [69]:
# prompt: Prompt : Write a query to display details of employees who have the 3rd highest salary in each department, if multiple emplouyees have the same salary as the 3rd highest in the department in a deaprtment, include all of them. the output should include department_id, employee_id, employee_name(fullname), salary

query = """
    SELECT department_id, employee_id, CONCAT(first_name, ' ', last_name) AS employee_name, salary
    FROM (
        SELECT
            department_id,
            employee_id,
            first_name,
            last_name,
            salary,
            DENSE_RANK() OVER (PARTITION BY department_id ORDER BY salary DESC) as salary_rank
        FROM
            `sathish-scaler-projects.employee_db.employees`
    ) ranked_salaries
    WHERE salary_rank = 3;
"""

query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
print(tabulate(df, headers='keys', tablefmt='psql'))


+----+-----------------+---------------+-------------------+----------+
|    |   department_id |   employee_id | employee_name     |   salary |
|----+-----------------+---------------+-------------------+----------|
|  0 |              80 |           147 | Alberto Errazuriz |    12000 |
|  1 |              50 |           122 | Payam Kaufling    |     7900 |
|  2 |              30 |           116 | Shelli Baida      |     2900 |
|  3 |              60 |           105 | David Austin      |     4800 |
|  4 |              60 |           106 | Valli Pataballa   |     4800 |
|  5 |             100 |           110 | John Chen         |     8200 |
+----+-----------------+---------------+-------------------+----------+


In [70]:
# prompt: Divde employees within each department into four equal salary groups (quartiles) , list each employees , employee_id, department_id, employee_name, salary and thier salary_quartile( indicating the quartile they fall into based in the salary within thier department, the 1st quartile should contain the employees with the highest salaries , the 4th quartile should contain the employees with lowest salaries

query = """
    WITH EmployeeSalaries AS (
        SELECT
            employee_id,
            department_id,
            first_name,
            last_name,
            salary,
            NTILE(4) OVER (PARTITION BY department_id ORDER BY salary DESC) AS salary_quartile
        FROM
            `sathish-scaler-projects.employee_db.employees`
    )
    SELECT
        employee_id,
        department_id,
        CONCAT(first_name, ' ', last_name) AS employee_name,
        salary,
        salary_quartile
    FROM
        EmployeeSalaries
    ORDER BY
        department_id,
        salary_quartile,
        salary DESC;
"""

query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
print(tabulate(df, headers='keys', tablefmt='psql'))


+-----+---------------+-----------------+-------------------+----------+-------------------+
|     |   employee_id | department_id   | employee_name     |   salary |   salary_quartile |
|-----+---------------+-----------------+-------------------+----------+-------------------|
|   0 |           200 | 10              | Jennifer Whalen   |     4400 |                 1 |
|   1 |           108 | 100             | Nancy Greenberg   |    12000 |                 1 |
|   2 |           109 | 100             | Daniel Faviet     |     9000 |                 1 |
|   3 |           110 | 100             | John Chen         |     8200 |                 2 |
|   4 |           112 | 100             | Jose Manuel Urman |     7800 |                 2 |
|   5 |           111 | 100             | Ismael Sciarra    |     7700 |                 3 |
|   6 |           113 | 100             | Luis Popp         |     6900 |                 4 |
|   7 |           205 | 110             | Shelley Higgins   |    12000

In [74]:
# prompt: Write a query to display each employee employee_id, department_id, salary , the salary of the next highest paid employee, and the difference in salary between each employee and the next highest paid employee in the company sort the results by salary and next_salary to show employees in ascending order of salary

query = """
    SELECT
        employee_id,
        department_id,
        salary,
        LEAD(salary, 1, 0) OVER (ORDER BY salary DESC) AS next_salary,
        salary - LEAD(salary, 1, 0) OVER (ORDER BY salary DESC) AS salary_difference
    FROM
        `sathish-scaler-projects.employee_db.employees`
    ORDER BY
        salary, next_salary;
"""
query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
print(tabulate(df, headers='keys', tablefmt='psql'))


+-----+---------------+-----------------+----------+---------------+---------------------+
|     |   employee_id | department_id   |   salary |   next_salary |   salary_difference |
|-----+---------------+-----------------+----------+---------------+---------------------|
|   0 |           132 | 50              |     2100 |             0 |                2100 |
|   1 |           136 | 50              |     2200 |          2100 |                 100 |
|   2 |           128 | 50              |     2200 |          2200 |                   0 |
|   3 |           135 | 50              |     2400 |          2200 |                 200 |
|   4 |           127 | 50              |     2400 |          2400 |                   0 |
|   5 |           191 | 50              |     2500 |          2400 |                 100 |
|   6 |           140 | 50              |     2500 |          2500 |                   0 |
|   7 |           131 | 50              |     2500 |          2500 |                   0 |

In [75]:
# prompt: Write a query to display each employee's department_id, employee_id, hire_date, salary, the salary of the first employee hired in their department, and the salary of the most recently hired employee in their department. Sort the result by department_id

query = """
    SELECT
        department_id,
        employee_id,
        hire_date,
        salary,
        FIRST_VALUE(salary) OVER (PARTITION BY department_id ORDER BY hire_date ASC) AS first_hire_salary,
        LAST_VALUE(salary) OVER (PARTITION BY department_id ORDER BY hire_date ASC RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_hire_salary
    FROM
        `sathish-scaler-projects.employee_db.employees`
    ORDER BY
        department_id;
"""
query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
print(tabulate(df, headers='keys', tablefmt='psql'))


+-----+-----------------+---------------+-------------+----------+---------------------+--------------------+
|     | department_id   |   employee_id | hire_date   |   salary |   first_hire_salary |   last_hire_salary |
|-----+-----------------+---------------+-------------+----------+---------------------+--------------------|
|   0 | 10              |           200 | 1987-09-17  |     4400 |                4400 |               4400 |
|   1 | 100             |           109 | 1994-08-16  |     9000 |                9000 |               6900 |
|   2 | 100             |           108 | 1994-08-17  |    12000 |                9000 |               6900 |
|   3 | 100             |           110 | 1997-09-28  |     8200 |                9000 |               6900 |
|   4 | 100             |           111 | 1997-09-30  |     7700 |                9000 |               6900 |
|   5 | 100             |           112 | 1998-03-07  |     7800 |                9000 |               6900 |
|   6 | 10

In [78]:
# prompt: Write a query to display each employee's employee_id, department_id, salary, hire_date, and two types of moving sums within each department, ordered by hire_date:
# mov_sum_1: The moving sum of salaries including the current row, the row before, and the row after. (with a window size of N=3) ,
# mov_sum_2: The moving sum of salaries including the current row and the two rows before it. (with a window size of N=3)

query = """
    SELECT
        employee_id,
        department_id,
        salary,
        hire_date,
        SUM(salary) OVER (PARTITION BY department_id ORDER BY hire_date ASC ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) AS mov_sum_1,
        SUM(salary) OVER (PARTITION BY department_id ORDER BY hire_date ASC ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS mov_sum_2
    FROM
        `sathish-scaler-projects.employee_db.employees`
    ORDER BY
        department_id,
        hire_date;
"""

query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
print(tabulate(df, headers='keys', tablefmt='psql'))


+-----+---------------+-----------------+----------+-------------+-------------+-------------+
|     |   employee_id | department_id   |   salary | hire_date   |   mov_sum_1 |   mov_sum_2 |
|-----+---------------+-----------------+----------+-------------+-------------+-------------|
|   0 |           200 | 10              |     4400 | 1987-09-17  |        4400 |        4400 |
|   1 |           109 | 100             |     9000 | 1994-08-16  |       21000 |        9000 |
|   2 |           108 | 100             |    12000 | 1994-08-17  |       29200 |       21000 |
|   3 |           110 | 100             |     8200 | 1997-09-28  |       27900 |       29200 |
|   4 |           111 | 100             |     7700 | 1997-09-30  |       23700 |       27900 |
|   5 |           112 | 100             |     7800 | 1998-03-07  |       22400 |       23700 |
|   6 |           113 | 100             |     6900 | 1999-12-07  |       14700 |       22400 |
|   7 |           205 | 110             |    12000